# Deploy VARCO TTS LITE from AWS Marketplace 


VARCO TTS LITE is a low-latency, high-throughput engine for large-scale real-time speech synthesis. It offers a wide range of voices optimized for in-game characters—NPCs, avatars, and story narration—while keeping audio quality stable and consistent across sessions.

To avoid issues often seen in AR (autoregressive) models, such as irregular pronunciation or instability in audio quality, the system tightly controls pitch, duration, and energy prediction, and applies deterministic synthesis to deliver the same quality every time.

This sample notebook shows you how to deploy [VARCO TTS LITE]()  using Amazon SageMaker.

> **Note**: This is a reference notebook and it cannot run unless you make changes suggested in the notebook.

## Pre-requisites:
1. **Note**: This notebook contains elements which render correctly in Jupyter interface. Open this notebook from an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
1. Ensure that IAM role used has **AmazonSageMakerFullAccess**
1. To deploy this ML model successfully, ensure that:
    1. Either your IAM role has these three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account used: 
        1. **aws-marketplace:ViewSubscriptions**
        1. **aws-marketplace:Unsubscribe**
        1. **aws-marketplace:Subscribe**  

## Contents:
1. [Subscribe to the model package](#1.-Subscribe-to-the-model-package)
2. [Create an endpoint and perform real-time inference](#2.-Create-an-endpoint-and-perform-real-time-inference)
   1. [Create an endpoint](#A.-Create-an-endpoint)
   2. [Create input payload](#B.-Create-input-payload)
   3. [Perform real-time inference](#C.-Perform-real-time-inference)
   4. [Delete the endpoint](#E.-Delete-the-endpoint)
3. [Clean-up](#3.-Clean-up)

  

## Usage instructions
You can run this notebook one cell at a time (By using Shift+Enter for running a cell).

## 1. Subscribe to the model package


To subscribe to the model package:

1. Open the model package listing page [VARCO TTS Lite]() Model.
2. On the AWS Marketplace listing, click on the **Continue to subscribe** button.
3. On the **Subscribe to this software** page, review and click on **"Accept Offer"** if you and your organization agrees with EULA, pricing, and support terms.
4. Once you click on **Continue to configuration button** and then choose a **region**, you will see a **Product Arn** displayed. This is the model package ARN that you need to specify while creating a deployable model using Boto3. Copy the ARN corresponding to your region and specify the same in the following cell.


In [ ]:
model_package_arn = "*"

In [ ]:
import json

import boto3
import sagemaker as sage
from sagemaker import ModelPackage, get_execution_role

In [ ]:
role = get_execution_role()

sagemaker_session = sage.Session()

bucket = sagemaker_session.default_bucket()
runtime = boto3.client("runtime.sagemaker")

## 2. Create an endpoint and perform real-time inference

If you want to understand how real-time inference with Amazon SageMaker works, see [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/how-it-works-hosting.html).


In [ ]:
endpoint_name = "varco-tts-lite-endpoint"
content_type = "application/json"
my_instance_type = "*"

### A. Create an endpoint

Once endpoint has been created, you would be able to perform real-time inference.

In [ ]:
# create a deployable model from the model package.
model = ModelPackage(role=role, model_package_arn=model_package_arn, sagemaker_session=sagemaker_session)

# Deploy the model
predictor = model.deploy(1, my_instance_type, endpoint_name=endpoint_name)

### B. Create input payload

- voice_id: Input the voice_id that you want to use. For more detailed information on available voices, see our [Voice List](voice-list.md)
- language: Specify the language of your input “text” using a combination of the desired language code (ISO 639-1) and country code(ISO 3166-1) separated by an underscore.
    - If the value isn’t supported by the model, an error is returned.
    - Supported languages: “ko_KR”(Korean), “en_US”(English), “ja_JP”(Japanese), “zh_TW”(Traditional Chinese, Taiwan).
- emotion: Choose the emotion to apply to the selected voice.
    - The supported emotions vary depending on which voice (i.e. “voice_id”) is selected. The emotion range
    - includes: "neutral", "angry", "happy", "sad", "fearful", "surprised".
    - If not provided, the default value is “neutral”.
    - If the selected voice does not support the requested emotion, an error is returned.
- text: Enter the text to synthesize.
    

In [ ]:
input = {
  "voice_id": "39251bb8-8cea-59f1-9f3b-e4f255b8875b",
  "language": "en_US",
  "emotion": "neutral",
  "text": "How are things with you lately? I’d love to hear what you’ve been up to."
}

### C. Perform real-time inference

##### C-1. Inference example

In [ ]:
runtime = boto3.client("sagemaker-runtime", region_name="us-east-1")
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType=content_type, Body=json.dumps(input))

result = response["Body"].read()

##### C-2. Listen to Synthesized Speech

In [ ]:
# Save audio to file
saved_path = "output/varco_tts_lite_sample.wav"
with open(saved_path, "wb") as f:
    f.write(result)

In [2]:
import IPython.display as ipd
ipd.Audio(saved_path)

### E. Delete the endpoint

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.


In [ ]:
model.sagemaker_session.delete_endpoint(endpoint_name)
model.sagemaker_session.delete_endpoint_config(endpoint_name)

## 3. Clean-up

Now that you have successfully performed a real-time inference, you do not need the endpoint any more. You can terminate the endpoint to avoid being charged.

In [ ]:
predictor.delete_model()
predictor.delete_endpoint()